In [1]:
import torch
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torch.utils.data as Data  
import torchvision 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#path = 'movielens-20m-dataset'
path = 'ml-latest-small'
rating = pd.read_csv("./"+path+"/rating.csv")
rating = np.array(rating)
print(1)
movie = pd.read_csv("./"+path+"/movie.csv")
movie = np.array(movie)
print(2)
tag = pd.read_csv("./"+path+"/tag.csv")
tag = np.array(tag)
print(3)
genome_scores = pd.read_csv("./"+path+"/genome_scores.csv")
genome_scores = np.array(genome_scores)
print(4)
genome_tags = pd.read_csv("./"+path+"/genome_tags.csv")
genome_tags = np.array(genome_tags)

1
2
3
4


### get nuser & nmovie

In [3]:
print("movie.shape",movie.shape)  
users1 = np.unique(rating[:,0])
print("users1.shape",users1.shape)  
users2 = np.unique(tag[:,0])
print("users2.shape",users2.shape)  
users = np.unique(np.hstack((users1,users2)))
print("users.shape",users.shape) 

movie.shape (9125, 3)
users1.shape (671,)
users2.shape (61,)
users.shape (671,)


In [4]:
nuser = users.shape[0]
nmovie = movie.shape[0]

In [5]:
X = torch.Tensor(nuser,nmovie, 5)  # | rate | mean_rate | tag | relevance | genre(int) |

In [7]:
matrix_rate = np.ones((rating.shape[0],2))
print(matrix_rate.shape)

(100004, 2)


In [15]:
#算mean_rate
print(rating.shape) #(20000263, 4)
for i in range(rating.shape[0]):
    movieId,rate = rating[i,1:3]
    movieId = movieId.astype(int)
    if(movieId <= nmovie and rate != str):
        matrix_rate[movieId-1,0] = matrix_rate[movieId-1,0] + rate
        matrix_rate[movieId-1,1] = matrix_rate[movieId-1,1] + 1
    if(i%1000000 == 0):
        print(i)

(100004, 4)
0


In [16]:
mean_rate = matrix_rate[:,0]/matrix_rate[:,1]
#mean_rate.shape #(20000263,)

In [52]:
#save user's rate
for i in range(rating.shape[0]):
    userId,movieId,rate = rating[i,0:3]
    movieId = movieId.astype(int)
    if(movieId <= nmovie and rate != str):   #有的rating里的movieId,MOVIE表里没有
        X[userId - 1,movieId - 1,0] = rate
    if(i%1000000 == 0):
        print(i)

0


In [18]:
#save mean_rate 巨慢。。。

for i in range(nmovie):
    X[:,i,1] = mean_rate[i]
    if(i%1000000 == 0):
        print(i)


0


In [ ]:
'''
X_file = X.numpy()
np.save("X_file.npy",X_file)
'''

In [69]:
#save user's tag： tensor里只能存同一种数据类型的变量
#超级慢。。
for i in range(tag.shape[0]):
    userId,movieId,tag_desc = tag[i,0:3]
    #判断tagId是否存在
    if(np.where(genome_tags[:,1] == tag_desc)[0].size != 0):
        tagId = np.where(genome_tags == tag_desc)[0][0]
        if(movieId <= nmovie ):   #有的rating里的movieId,MOVIE表里没有
            X[userId - 1,movieId - 1,2] = float(tagId)
            '''for j in range(genome_scores.shape[0]):
                if(genome_scores[j,0] == movieId and genome_scores[j,1] == tagId):
                    X[userId - 1,movieId - 1,3] = genome_scores[j,2]'''
    if(i%10000 == 0):
        print(i)


0


KeyboardInterrupt: 

In [93]:
genre_list = np.array(['Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'])

In [95]:
#save genre
for i in range(movie.shape[0]):
    movieId, genres = movie[i,(0,2)]
    genres_split = genres.split('|')
    for genre in genres_split:
        x = 0
        index = np.where(genre_list ==genre)[0]
        if(index.size > 0):
            x = x + 2**index
    if(movieId <= nmovie ):
        X[:,movieId - 1,4] = float(x)

In [75]:
genres_split = genres.split('|')

In [84]:
genre_list = np.array(['Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'])